In [31]:
import json
import os
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#Para entrenamiento
path_to_json = './'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

In [32]:
#Para Test
path_to_json = 'C://Users//daoban//OneDrive//Tesis_QEC//SimulacionCircuitos//gen_sampler//p0_1'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

In [33]:
json_files = json_files[:len(json_files)-1] #xq ya hay una json

In [6]:
json_text.keys()

dict_keys(['X_corrections', 'X_stab_outcomes', 'Z_corrections', 'Z_stab_outcomes', 'error_added', 'failings', 'failings_total_NN', 'final_errors', 'n_correctable', 'n_fail', 'n_fail_NN', 'n_supra_gates', 'n_total', 'p_1q', 'p_2q', 'p_correctable', 'p_fail', 'p_fail_NN', 'p_meas', 'p_supra_gates'])

In [34]:
x_correc = []
x_stab = []
z_correc = []
z_stab = []
final_errors = []
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)
        x_correc.append(json_text['X_corrections'])
        x_stab.append(json_text['X_stab_outcomes'])
        z_correc.append(json_text['Z_corrections'])
        z_stab.append(json_text['Z_stab_outcomes'])
        final_errors.append(json_text['failings_total_NN'])

In [35]:
#relleno los que tienen solo dos mediciones iguales x
for i in range(0,len(x_stab)):
    for j in range(0,len(x_stab[i])):
        if len(x_stab[i][j]) != 3:
           x_stab[i][j].append(x_stab[i][j][0])

In [36]:
#relleno los que tienen solo dos mediciones iguales z
for i in range(0,len(z_stab)):
    for j in range(0,len(z_stab[i])):
        if len(z_stab[i][j]) != 3:
           z_stab[i][j].append(z_stab[i][j][0])

In [37]:
dataz_stab =[]
for i in range(0,len(z_stab)):
    for j in range(0,len(z_stab[i])):
        for k in range(0,len(z_stab[i][j])):
            for l in range(0,len(z_stab[i][j][k])):
                dataz_stab.append(z_stab[i][j][k][l])

In [38]:
datax_stab =[]
for i in range(0,len(x_stab)):
    for j in range(0,len(x_stab[i])):
        for k in range(0,len(x_stab[i][j])):
            for l in range(0,len(x_stab[i][j][k])):
                datax_stab.append(x_stab[i][j][k][l])

In [39]:
numeric = []
for i in range(0,len(final_errors)):
    for j in range(0,len(final_errors[i])):
        numeric.append(final_errors[i][j])
numeric = np.array(numeric).astype(int)

In [40]:
datax_stab = np.array(datax_stab).reshape((len(json_files)*len(json_text['X_corrections']),12))
dataz_stab = np.array(dataz_stab).reshape((len(json_files)*len(json_text['X_corrections']),12))

indices = []
for index, (vectx, vectz) in enumerate(zip(datax_stab,dataz_stab)):
        if max(vectx)==0 & max(vectz)==0:
            indices.append(index)
delete = indices#[:corte]
datax_stab=np.delete(datax_stab,delete,axis=0)
dataz_stab=np.delete(dataz_stab,delete,axis=0)
numeric = np.delete(numeric,delete,axis=0)
datax3 = np.array(datax_stab).reshape(len(datax_stab)*3,4)
dataz3 = np.array(dataz_stab).reshape(len(datax_stab)*3,4)
numeric3 = []
for i in range(0,len(datax_stab)):
    for j in range(0,3):
        numeric3.append(numeric[i])
numeric3 = np.array(numeric3)
data3_stab = np.concatenate((datax3,dataz3),axis = 1)
data3 = np.concatenate((data3_stab,numeric3.reshape(len(datax_stab)*3,1)),axis=1)

In [41]:
indices = []
for index, (vectx, vectz) in enumerate(zip(datax_stab,dataz_stab)):
        if max(vectx)==0 & max(vectz)==0:
            indices.append(index)

In [42]:
delete = indices#[:corte]

In [43]:
corte =len(datax_stab)-len(indices)

In [44]:
datax_stab=np.delete(datax_stab,delete,axis=0)
dataz_stab=np.delete(dataz_stab,delete,axis=0)
numeric = np.delete(numeric,delete,axis=0)

In [45]:
datax3 = np.array(datax_stab).reshape(len(datax_stab)*3,4)
dataz3 = np.array(dataz_stab).reshape(len(datax_stab)*3,4)

In [46]:
numeric3 = []
for i in range(0,len(datax_stab)):
    for j in range(0,3):
        numeric3.append(numeric[i])
numeric3 = np.array(numeric3)

In [47]:
data3_stab = np.concatenate((datax3,dataz3),axis = 1)
data3 = np.concatenate((data3_stab,numeric3.reshape(len(datax_stab)*3,1)),axis=1)

In [48]:
np.savetxt('training_p01.csv',data3,delimiter=",")

In [49]:
def separar_secuencia(secuencia,pasos_tiempo):
    x, y = list(), list()
    for i in range(len(secuencia)):
        final_ix = i +pasos_tiempo
        if final_ix > len(secuencia)-1:
            break
        #juntamos la serie y resultado del siguiente paso
        seq_x, seq_y = secuencia[i:final_ix,:-1], secuencia[final_ix-1,-1]
        x.append(seq_x)
        y.append(seq_y)
    return np.array(x), np.array(y)

In [23]:
pasos_tiempo3 = 3
x3_0, y3_0 = separar_secuencia(data3,pasos_tiempo3)

In [24]:
x3 = []
y3 = []
for index in range(0,len(x3_0)):
    if index==0:
        x3.append(x3_0[index])
    elif index%3 == 0:
        x3.append(x3_0[index])
for index in range(0,len(y3_0)):
    if index==0:
        y3.append(y3_0[index])
    elif index%3 == 0:
        y3.append(y3_0[index])
x3 = np.array(x3)
y3 = np.array(y3)

In [25]:
indices = np.arange(x3.shape[0])

In [26]:
np.random.shuffle(indices)

In [27]:
x3 = x3[indices]
y3 = y3[indices]

In [28]:
x_train3 = x3[0:round(len(x3)*0.8)]
y_train3 = y3[0:round(len(y3)*0.8)]
x_test3 = x3[round(len(x3)*0.8):len(x3)]
y_test3 = y3[round(len(y3)*0.8):len(y3)]

In [29]:
y_train3= keras.utils.to_categorical(y_train3,num_classes=2)
y_test3 = keras.utils.to_categorical(y_test3,num_classes=2)

In [30]:
n_stab3 = 8

In [40]:
model3 = Sequential()
model3.add(LSTM(128,
               activation = 'relu',
               return_sequences = True,
               input_shape = (pasos_tiempo3,n_stab3)))
model3.add(LSTM(64, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(64,activation = 'relu'))
model3.add(Dropout(0.3))
model3.add(Dense(32,activation = 'relu'))
model3.add(Dropout(0.3))
model3.add(Dense(16,activation = 'relu'))
model3.add(Dense(2, activation='softmax'))

In [41]:
model3.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [42]:
model3.fit(x_train3, y_train3,batch_size=300, epochs=20,verbose=1)

Epoch 1/20


InternalError: GPU sync failed

In [ ]:
model3.save('modelo.h5')

In [72]:
score3 = model3.evaluate(x_test3, y_test3, batch_size=256)
score3

324000/324000 [==============================] - 9s 29us/step


[0.5007690983171816, 0.7523765431981028]

In [3]:
from keras.models import load_model

In [4]:
model3 = load_model('modelo.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [5]:
texto = """x_correc_02e = []
x_stab_02e = []
z_correc_02e = []
z_stab_02e = []
final_errors_02e = []
with open(path_to_json+'0_2.json') as json_file:
    json_text = json.load(json_file)
x_correc_02e.append(json_text['X_corrections'])
x_stab_02e.append(json_text['X_stab_outcomes'])
z_correc_02e.append(json_text['Z_corrections'])
z_stab_02e.append(json_text['Z_stab_outcomes'])
final_errors_02e.append(json_text['failings_total_NN'])
for i in range(0,len(x_stab_02e)):
    for j in range(0,len(x_stab_02e[i])):
        if len(x_stab_02e[i][j]) != 3:
           x_stab_02e[i][j].append(x_stab_02e[i][j][0])
#relleno los que tienen solo dos mediciones iguales z
for i in range(0,len(z_stab_02e)):
    for j in range(0,len(z_stab_02e[i])):
        if len(z_stab_02e[i][j]) != 3:
           z_stab_02e[i][j].append(z_stab_02e[i][j][0])
dataz_stab_02e =[]
for i in range(0,len(z_stab_02e)):
    for j in range(0,len(z_stab_02e[i])):
        for k in range(0,len(z_stab_02e[i][j])):
            for l in range(0,len(z_stab_02e[i][j][k])):
                dataz_stab_02e.append(z_stab_02e[i][j][k][l])
datax_stab_02e =[]
for i in range(0,len(x_stab_02e)):
    for j in range(0,len(x_stab_02e[i])):
        for k in range(0,len(x_stab_02e[i][j])):
            for l in range(0,len(x_stab_02e[i][j][k])):
                datax_stab_02e.append(x_stab_02e[i][j][k][l])
datax_stab_02e = np.array(datax_stab_02e).reshape((30000,12))
dataz_stab_02e = np.array(dataz_stab_02e).reshape((30000,12))
numeric_02e = []
for i in range(0,len(final_errors_02e[0])):
    for j in range(0,3):
        numeric_02e.append(final_errors_02e[0][i])
numeric_02e = np.array(numeric_02e).astype(int)
indices = []
for index, (vectx, vectz) in enumerate(zip(datax_stab_02e,dataz_stab_02e)):
        if max(vectx)==0 & max(vectz)==0:
            indices.append(index)
delete_02e = indices#[:corte]
datax_stab_02e=np.delete(datax_stab_02e,delete_02e,axis=0)
dataz_stab_02e=np.delete(dataz_stab_02e,delete_02e,axis=0)
numeric_02e = np.delete(numeric_02e,delete_02e,axis=0)
datax3 = np.array(datax_stab_02e).reshape(len(datax_stab_02e)*3,4)
dataz3 = np.array(dataz_stab_02e).reshape(len(datax_stab_02e)*3,4)
numeric3 = []
for i in range(0,len(datax_stab_02e)):
    for j in range(0,3):
        numeric3.append(numeric_02e[i])
numeric3 = np.array(numeric3)
data3_stab = np.concatenate((datax3,dataz3),axis = 1)
data3 = np.concatenate((data3_stab,numeric3.reshape(len(datax_stab_02e)*3,1)),axis=1)
pasos_tiempo3 = 3
x3_0, y3_0 = separar_secuencia(data3,pasos_tiempo3)
x3 = []
y3 = []
for index in range(0,len(x3_0)):
    if index==0:
        x3.append(x3_0[index])
    elif index%3 == 0:
        x3.append(x3_0[index])
for index in range(0,len(y3_0)):
    if index==0:
        y3.append(y3_0[index])
    elif index%3 == 0:
        y3.append(y3_0[index])
x3 = np.array(x3)
x3=x3[round(len(x3)*0.8):len(x3)]
y3 = np.array(y3)
y3=y3[round(len(y3)*0.8):len(y3)]
y3= keras.utils.to_categorical(y3,num_classes=2)
acc_02e = model3.evaluate(x3, y3, batch_size=256)[1]
delete_02e = len(delete_02e)
acc_02e"""

In [6]:
files = []
for file in json_files:
    files.append(file.replace('.json',''))

In [7]:
numbers = []
for n in files:
    numbers.append(n.replace('_',''))

In [10]:
for i,j in zip(files,numbers):
    print(i,j)
    t1 = texto.replace('0_2',i)
    string = j+'e'
    t2 = t1.replace('02e',string)
    exec(t2)

0_1 01
2055/2055 [==============================] - 1s 304us/step
0_2 02
3317/3317 [==============================] - 0s 28us/step
0_3 03
4144/4144 [==============================] - 0s 26us/step
0_4 04
4674/4674 [==============================] - 0s 27us/step
0_5 05
5065/5065 [==============================] - 0s 28us/step
0_6 06
5296/5296 [==============================] - 0s 24us/step
0_7 07
5482/5482 [==============================] - 0s 26us/step
0_8 08
5579/5579 [==============================] - 0s 31us/step
0_9 09
5681/5681 [==============================] - 0s 27us/step
1_0 10
2003/2003 [==============================] - 0s 31us/step
1_1 11
3319/3319 [==============================] - 0s 33us/step
1_2 12
4171/4171 [==============================] - 0s 26us/step
1_3 13
4720/4720 [==============================] - 0s 26us/step
1_4 14
5081/5081 [==============================] - 0s 28us/step
1_5 15
5317/5317 [==============================] - 0s 26us/step
1_6 16
5476/5476 [======

In [11]:
acc = np.array([acc_01e,acc_02e,acc_03e,acc_04e,acc_05e,acc_06e,acc_07e,acc_08e,acc_09e,acc_10e,acc_11e,acc_12e,acc_13e,acc_14e,acc_15e,acc_16e,acc_17e,acc_18e,acc_20e,acc_21e,acc_22e,acc_23e,acc_24e,acc_25e,acc_26e,acc_27e,acc_30e,acc_31e,acc_32e,acc_33e,acc_34e,acc_35e,acc_36e,acc_40e,acc_41e,acc_42e,acc_43e,acc_44e,acc_45e,acc_50e,acc_51e,acc_52e,acc_53e,acc_54e,acc_60e,acc_61e,acc_62e,acc_63e,acc_70e,acc_71e,acc_72e,acc_80e,acc_81e,acc_90e])

In [12]:
delete = np.array([delete_01e,delete_02e,delete_03e,delete_04e,delete_05e,delete_06e,delete_07e,delete_08e,delete_09e,delete_10e,delete_11e,delete_12e,delete_13e,delete_14e,delete_15e,delete_16e,delete_17e,delete_18e,delete_20e,delete_21e,delete_22e,delete_23e,delete_24e,delete_25e,delete_26e,delete_27e,delete_30e,delete_31e,delete_32e,delete_33e,delete_34e,delete_35e,delete_36e,delete_40e,delete_41e,delete_42e,delete_43e,delete_44e,delete_45e,delete_50e,delete_51e,delete_52e,delete_53e,delete_54e,delete_60e,delete_61e,delete_62e,delete_63e,delete_70e,delete_71e,delete_72e,delete_80e,delete_81e,delete_90e])

In [13]:
delete

array([19724, 13413,  9280,  6630,  4676,  3518,  2587,  2102,  1594,
       19986, 13406,  9146,  6401,  4596,  3416,  2620,  1896,  1565,
       13038,  8982,  6275,  4443,  3298,  2422,  1862,  1492,  8584,
        6070,  4298,  3110,  2385,  1822,  1414,  5762,  4210,  3061,
        2337,  1703,  1389,  3913,  2940,  2089,  1622,  1293,  2699,
        2020,  1590,  1184,  1908,  1421,  1120,  1319,  1127,  1006])

In [14]:
p = ['acc_01e','acc_02e','acc_03e','acc_04e','acc_05e','acc_06e','acc_07e','acc_08e','acc_09e','acc_10e','acc_11e','acc_12e','acc_13e','acc_14e','acc_15e','acc_16e','acc_17e','acc_18e','acc_20e','acc_21e','acc_22e','acc_23e','acc_24e','acc_25e','acc_26e','acc_27e','acc_30e','acc_31e','acc_32e','acc_33e','acc_34e','acc_35e','acc_36e','acc_40e','acc_41e','acc_42e','acc_43e','acc_44e','acc_45e','acc_50e','acc_51e','acc_52e','acc_53e','acc_54e','acc_60e','acc_61e','acc_62e','acc_63e','acc_70e','acc_71e','acc_72e','acc_80e','acc_81e','acc_90e']
#strings = []
"""for x in p:
    strings.append(str(x))
strings = np.array(strings)"""

'for x in p:\n    strings.append(str(x))\nstrings = np.array(strings)'

In [15]:
peso_cero = ['delete_01e','delete_02e','delete_03e','delete_04e','delete_05e','delete_06e','delete_07e','delete_08e','delete_09e','delete_10e','delete_11e','delete_12e','delete_13e','delete_14e','delete_15e','delete_16e','delete_17e','delete_18e','delete_20e','delete_21e','delete_22e','delete_23e','delete_24e','delete_25e','delete_26e','delete_27e','delete_30e','delete_31e','delete_32e','delete_33e','delete_34e','delete_35e','delete_36e','delete_40e','delete_41e','delete_42e','delete_43e','delete_44e','delete_45e','delete_50e','delete_51e','delete_52e','delete_53e','delete_54e','delete_60e','delete_61e','delete_62e','delete_63e','delete_70e','delete_71e','delete_72e','delete_80e','delete_81e','delete_90e']

In [16]:
fail = 1-acc

In [17]:
output_filename = 'summary_nn_p01.json'
output_dict = {}
for i in range(len(acc)):
    output_dict[str((int(numbers[i][0]),int(numbers[i][1])))] = fail[i]
output_json_file = open(output_filename, 'w')
json.dump(output_dict, output_json_file)
output_json_file.close()

In [18]:
p_cero = []
for name, number in zip(peso_cero,delete):
    p_cero.append([name,number])

In [19]:
final = []
for name, number in zip(p,fail):
    final.append([name,number])

In [20]:
with open("results_30mil.txt", 'w') as output:
    for row in final:
        output.write(str(row) + '\n')

In [21]:
with open("peso_cero.txt", 'w') as output:
    for row in p_cero:
        output.write(str(row) + '\n')